In [5]:
# reporting/quarterly_report/modules/granting.py
from __future__ import annotations

import logging, sqlite3, datetime
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
from great_tables import GT, loc, style, html

# our project
from ingestion.db_utils import (
    init_db,                                 # create tables if missing
    fetch_latest_table_data,                 # new version!
    get_alias_last_load,
    get_variable_status,                     # to inspect results
)

from reporting.quarterly_report.utils import RenderContext, BaseModule
from reporting.quarterly_report.report_utils.granting_utils import enrich_grants, _ensure_timedelta_cols, _coerce_date_columns
from reporting.quarterly_report.utils import Database, RenderContext

# ─────────────────────────────────────────────────────────────
# 2) open DB – change path if you work on a copy
# ─────────────────────────────────────────────────────────────
DB_PATH = Path("database/reporting.db")
init_db(db_path=DB_PATH)            # no-op if tables already exist

db = Database(str(DB_PATH))         # thin sqlite3 wrapper
conn = db.conn


# ──────────────────────────────────────────────────────────────
# constants – adapt whenever a file-alias changes
# ──────────────────────────────────────────────────────────────

PO_SUMMA_ALIAS = "c0_po_summa"
cutoff = pd.to_datetime("2025-04-15")

df_summa  = fetch_latest_table_data(conn, PO_SUMMA_ALIAS,   cutoff)



CALLS_TYPES_LIST = ['STG','ADG','POC','COG','SYG','StG','CoG','AdG','SyG', 'PoC']
PROGRAMMES_LIST = ['HORIZONEU_21_27', 'H2020_14_20']
FUND_SOURCES = ['VOBU', 'EARN/N', 'EFTA' , 'IAR2/2']

df_summa_filtered = df_summa[df_summa['Functional Area'].isin(PROGRAMMES_LIST)]
df_summa_filtered = df_summa_filtered[df_summa_filtered['Fund Source'].isin(FUND_SOURCES)]

# Function to determine PO_CATEGORY based on the rules
def determine_po_category(row):
    po_category_desc = str(row.get('PO Category Desc', '')).strip()
    po_abac_sap_ref = str(row.get('PO ABAC SAP Reference', '')).strip()
    po_purchase_order_desc = str(row.get('PO Purchase Order Desc', '')).strip()

    if po_category_desc == 'Grant':
        # Check PO ABAC SAP Reference first
        if po_abac_sap_ref and any(call_type in po_abac_sap_ref for call_type in CALLS_TYPES_LIST):
            return next(call_type for call_type in CALLS_TYPES_LIST if call_type in po_abac_sap_ref).upper()
        # If empty or no match, check PO Purchase Order Desc
        elif po_purchase_order_desc and any(call_type in po_purchase_order_desc for call_type in CALLS_TYPES_LIST):
            return next(call_type for call_type in CALLS_TYPES_LIST if call_type in po_purchase_order_desc).upper()
        return 'CSA/SJI'  # Return empty if no match found
    elif po_category_desc in ['Direct Contract', 'Specific Contract']:
        return 'Experts'
    return ''  # Default value for other cases

# Ensure df_summa_filtered is a new DataFrame to avoid SettingWithCopyWarning
df_summa_filtered = df_summa_filtered.copy()

# Apply the function to create the new column using .loc
df_summa_filtered.loc[:, 'PO_CATEGORY'] = df_summa_filtered.apply(determine_po_category, axis=1)

# Define the mapping dictionary
programme_mapping = {
    'HORIZONEU_21_27': 'HE',
    'H2020_14_20': 'H2020'
}

# Create the new column 'Programme' based on 'Functional Area'
df_summa_filtered['Programme'] = df_summa_filtered['Functional Area'].map(programme_mapping).fillna('')

# Perform aggregation by PO Purchase Order Key
aggregated_df = df_summa_filtered.groupby('PO Purchase Order Key').agg({
    'PO Open Amount - RAL - Payments Made (PD Approved)': 'sum',  # Sum the numeric column
    'Programme': 'first',  # Take the first non-null value (assuming consistency)
    'PO_CATEGORY': 'first',  # Take the first non-null value (assuming consistency)
    'PO Final Date of Implementation (dd/mm/yyyy)': 'max'  # Take the maximum (latest) date
}).reset_index()

# Rename the aggregated column for clarity (optional)
aggregated_df = aggregated_df.rename(columns={
    'PO Open Amount - RAL - Payments Made (PD Approved)': 'Total_Open_Amount',
    'PO Final Date of Implementation (dd/mm/yyyy)': 'PO Final Date of Implementation'
})

# Filter to keep only rows where Total_Open_Amount > 0
aggregated_df = aggregated_df[aggregated_df['Total_Open_Amount'] > 0]

# Pivot the agg_result to align with the table structure
pivot_open = pd.pivot_table(
    aggregated_df,
    index=['Programme','PO_CATEGORY'],
    #columns='Programme',
    values=['PO Purchase Order Key'],
    aggfunc="count",
    fill_value=0
).reset_index()

pivot_open.columns = [
    "Programme",
    "PO Type",
    "Total Commitments with RAL",
]

# # Compute total commitments with RAL by PO_CATEGORY for the filtered data
# total_ral_by_category = aggregated_df.groupby('PO_CATEGORY')['PO Purchase Order Key'].count().reset_index()
# total_ral_by_category = total_ral_by_category.rename(columns={'PO Purchase Order Key': 'Total commitments with RAL'})

# Ensure Latest_Validity_Period_End is in datetime format after aggregation
aggregated_df['PO Final Date of Implementation'] = pd.to_datetime(
    aggregated_df['PO Final Date of Implementation'], 
    format='%Y-%m-%d %H:%M:%S',  # Match the format from the table
    errors='coerce'
)

# Filter to keep only rows where Latest_Validity_Period_End <= cutoff
aggregated_df = aggregated_df[
    aggregated_df['PO Final Date of Implementation'].notna() & 
    (aggregated_df['PO Final Date of Implementation'] <= cutoff)
]
# Compute the number of days elapsed from Latest_Validity_Period_End to cutoff
aggregated_df['Days_Elapsed_From_Cutoff'] = (cutoff - aggregated_df['PO Final Date of Implementation']).dt.days

# Categorize based on Days_Elapsed_From_Cutoff
def categorize_days(days):
    if 0 <= days <= 60:
        return "Within 2 months"
    elif 61 <= days <= 180:
        return "Between 2 and 6 months"
    elif days > 180:
        return "More than 6 months"
    else:
        return "Overdue"  # Should not occur due to the <= cutoff filter

aggregated_df['Category'] = aggregated_df['Days_Elapsed_From_Cutoff'].apply(categorize_days)

# Aggregate by PO_CATEGORY
agg_result = aggregated_df.groupby(['Programme','PO_CATEGORY', 'Category']).agg({
    'PO Purchase Order Key': 'count',  # Count of items
   
}).reset_index()
# Merge the total commitments with RAL into the aggregated result

# Pivot the agg_result to align with the table structure
pivot_result = pd.pivot_table(
    agg_result,
    index=['Programme','PO_CATEGORY'],
    columns='Category',
    values=['PO Purchase Order Key'],
    aggfunc="sum",
    fill_value=0
).reset_index()

# Flatten MultiIndex columns
pivot_result.columns = ['__'.join(col).strip() if isinstance(col, tuple) else col for col in pivot_result.columns]


# Reset index
pivot_result = pivot_result.reset_index()

# Rename the columns to cleaner names
pivot_result.columns = [
    "Index",
    "Programme",
    "PO Type",
    "Between 2 and 6 Months",
    "More Than 6 Months",
    "Within 2 Months"
]

pivot_result.drop(columns=["Index"], inplace=True)

pivot_result = pivot_result[["Programme", "PO Type", "Within 2 Months", "Between 2 and 6 Months", "More Than 6 Months"]]

merged_df = pd.merge(pivot_result, pivot_open, on=["Programme", "PO Type"], how="outer")
merged_df = merged_df.fillna(0)

merged_df['Total Overdue'] = merged_df['More Than 6 Months'] + merged_df['Between 2 and 6 Months'] + merged_df['Within 2 Months']

merged_df["% of Overdue/running grants"] = merged_df['Total Overdue'] / merged_df["Total Commitments with RAL"] 
merged_df = merged_df[["Programme", "PO Type", "Within 2 Months", "Between 2 and 6 Months", "More Than 6 Months", 'Total Overdue', "% of Overdue/running grants", "Total Commitments with RAL"]]

merged_df 

DEBUG:root:Fetching latest data for table_alias: c0_po_summa, cutoff: 2025-04-15T00:00:00
DEBUG:root:Upload log query results for c0_po_summa: [('2025-05-14T08:34:51.323421', 11)]
DEBUG:root:Checking upload_id: 11, uploaded_at: 2025-05-14T08:34:51.323421
DEBUG:root:Fetched 15927 rows from c0_po_summa with upload_id 11


,Programme,PO Type,Within 2 Months,Between 2 and 6 Months,More Than 6 Months,Total Overdue,% of Overdue/running grants,Total Commitments with RAL
0,H2020,ADG,0.0,0.0,0.0,0.0,0.000000,805
1,H2020,COG,1.0,1.0,0.0,2.0,0.001883,1062
2,H2020,CSA/SJI,0.0,0.0,0.0,0.0,0.000000,1
3,H2020,POC,0.0,1.0,0.0,1.0,0.500000,2
4,H2020,STG,0.0,0.0,1.0,1.0,0.000809,1236
5,H2020,SYG,0.0,0.0,0.0,0.0,0.000000,98
6,HE,ADG,0.0,0.0,0.0,0.0,0.000000,726
7,HE,COG,0.0,0.0,0.0,0.0,0.000000,1168
8,HE,CSA/SJI,0.0,0.0,0.0,0.0,0.000000,2
9,HE,Experts,0.0,0.0,0.0,0.0,0.000000,30


In [9]:
merged_df

,Programme,PO Type,Within 2 Months,Between 2 and 6 Months,More Than 6 Months,Total Overdue,% of Overdue/running grants,Total Commitments with RAL
0,H2020,ADG,0.0,0.0,0.0,0.0,0.000000,805
1,H2020,COG,1.0,1.0,0.0,2.0,0.001883,1062
2,H2020,CSA/SJI,0.0,0.0,0.0,0.0,0.000000,1
3,H2020,POC,0.0,1.0,0.0,1.0,0.500000,2
4,H2020,STG,0.0,0.0,1.0,1.0,0.000809,1236
5,H2020,SYG,0.0,0.0,0.0,0.0,0.000000,98
6,HE,ADG,0.0,0.0,0.0,0.0,0.000000,726
7,HE,COG,0.0,0.0,0.0,0.0,0.000000,1168
8,HE,CSA/SJI,0.0,0.0,0.0,0.0,0.000000,2
9,HE,Experts,0.0,0.0,0.0,0.0,0.000000,30


In [10]:

# Compute totals for each Programme, excluding '% of Overdue/running grants'
numeric_cols = ['Within 2 Months', 'Between 2 and 6 Months', 'More Than 6 Months', 
                'Total Overdue', 'Total Commitments with RAL']

# Group by Programme and sum the numeric columns
totals = merged_df.groupby('Programme')[numeric_cols].sum().reset_index()

# Calculate '% of Overdue/running grants' for the total rows
totals['% of Overdue/running grants'] = totals['Total Overdue'] / totals['Total Commitments with RAL']
totals['% of Overdue/running grants'] = totals['% of Overdue/running grants'].fillna(0)  # Handle division by zero

# Create total rows with "PO Type" as "Total <Programme>"
total_rows = []
for _, row in totals.iterrows():
    programme = row['Programme']
    total_row = row.copy()
    total_row['PO Type'] = f'Total {programme}'
    total_rows.append(total_row)

# Convert total rows to DataFrame
total_df = pd.DataFrame(total_rows)

# Concatenate the original DataFrame with the total rows
df_with_totals = pd.concat([merged_df, total_df], ignore_index=True)

# Sort by Programme and then by PO Type to ensure totals appear at the end of each group
df = df_with_totals.sort_values(by=['Programme', 'PO Type']).reset_index(drop=True)

from great_tables import GT, loc, style

# Define colors
BLUE        = "#004A99"
LIGHT_BLUE =   "#d6e6f4"
GRID_CLR    = "#004A99"
DARK_BLUE   = "#01244B"
DARK_GREY =   '#242425'

# Define columns to display in the table (excluding 'Programme' and 'Type')
display_columns = df_with_totals.columns[2:-1].tolist()  # Exclude 'Programme', 'PO Type', and 'Type'

# Create the great table
if not df_with_totals.empty:
    tbl = (
    GT(
        df ,
        rowname_col="PO Type",
        groupname_col="Programme"
    )
    .tab_header(
        title="PO Purchase Orders exceeding the Final Date of Implementation"
    )
    # Format "numbers" group as integers (except percentage column)
    .fmt_number(
        columns=[col for col in display_columns if col != '% of Overdue/running grants'],
        # rows=df.index[df["Type"] == "numbers"].tolist(),
        decimals=0,
        use_seps=True
    )
    # Format '% of Overdue/running grants' as percentage with 2 decimal places
    .fmt_percent(
        columns='% of Overdue/running grants',
        # rows=df.index[df["Type"] == "numbers"].tolist(),
        decimals=1,
    )
    .tab_style(
        style.text(color=DARK_BLUE, weight="bold", align="center", font='Arial'),
        locations=loc.header()
    )
 
    .tab_stubhead(label="PO Type")
    .tab_style(
        style=[
            style.text(color=DARK_BLUE, weight="bold", font='Arial', size='medium'),
            style.fill(color=LIGHT_BLUE),
            style.css(f"border-bottom: 2px solid {DARK_BLUE}; border-right: 2px solid {DARK_BLUE}; border-top: 2px solid {DARK_BLUE}; border-left: 2px solid {DARK_BLUE};"),
            style.css("max-width:200px; line-height:1.2"),
        ],
        locations=loc.row_groups()
    )
    .opt_table_font(font="Arial")
    .tab_style(
        style=[
            # style.fill(color=BLUE),
            style.text(color="white", weight="bold", align="center", size='small'),
            style.css("max-width:200px; line-height:1.2")
        ],
        locations=loc.column_labels()
    )
    .tab_style(
        style=[
            # style.fill(color=BLUE),
            style.text(color="white", weight="bold", align="center", size='small'),
            style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2")
        ],
        locations=loc.stubhead()
    )
    .tab_style(
        style=[style.borders(weight="1px", color=DARK_BLUE),
               style.text(size='small')],
        locations=loc.stub()
    )
    .tab_style(
        style=[style.borders(sides="all", color=DARK_BLUE, weight="1px"),
               style.text(align="center", size='small')],
        locations=loc.body()
    )
    .tab_style(
        style=style.borders(color=DARK_BLUE, weight="2px"),
        locations=[loc.column_labels(), loc.stubhead()]
    )
    .tab_style(
        style=[style.fill(color="#D3D3D3"), style.text(color="black", weight="bold")],
        locations=loc.body(rows=df.index[df["PO Type"].str.contains("Total")].tolist())
    )
    .tab_style(
        style=[style.fill(color="#D3D3D3"), style.text(color="black", weight="bold")],
        locations=loc.stub(rows=df.index[df["PO Type"].str.contains("Total")].tolist())
    )
    .tab_options(
        table_body_border_bottom_color=DARK_BLUE,
        table_body_border_bottom_width="2px",
        table_border_right_color=DARK_BLUE,
        table_border_right_width="2px",
        table_border_left_color=DARK_BLUE,
        table_border_left_width="2px",
        table_border_top_color=DARK_BLUE,
        table_border_top_width="2px",
        column_labels_border_top_color=DARK_BLUE,
        column_labels_border_top_width="2px",
        heading_background_color=BLUE,
        row_group_background_color=BLUE

   
    )
    .tab_source_note("Source: Compass")
    .tab_source_note("Reports: Call Overview Report - Budget Follow-Up Report - Ethics Requirements and Issues")
    .tab_style(
        style=[style.text(size="small")],
        locations=loc.footer()
    )
)
else:
    print("No data to display.")

In [11]:
df_with_totals

,Programme,PO Type,Within 2 Months,Between 2 and 6 Months,More Than 6 Months,Total Overdue,% of Overdue/running grants,Total Commitments with RAL
0,H2020,ADG,0.0,0.0,0.0,0.0,0.000000,805
1,H2020,COG,1.0,1.0,0.0,2.0,0.001883,1062
2,H2020,CSA/SJI,0.0,0.0,0.0,0.0,0.000000,1
3,H2020,POC,0.0,1.0,0.0,1.0,0.500000,2
4,H2020,STG,0.0,0.0,1.0,1.0,0.000809,1236
5,H2020,SYG,0.0,0.0,0.0,0.0,0.000000,98
6,HE,ADG,0.0,0.0,0.0,0.0,0.000000,726
7,HE,COG,0.0,0.0,0.0,0.0,0.000000,1168
8,HE,CSA/SJI,0.0,0.0,0.0,0.0,0.000000,2
9,HE,Experts,0.0,0.0,0.0,0.0,0.000000,30


In [12]:
from great_tables import GT, loc, style

# Define colors
BLUE        = "#004A99"
LIGHT_BLUE =   "#d6e6f4"
GRID_CLR    = "#004A99"
DARK_BLUE   = "#01244B"
DARK_GREY =   '#242425'

# Define columns to display in the table (excluding 'Programme' and 'Type')
display_columns = df_with_totals.columns[2:-1].tolist()  # Exclude 'Programme', 'PO Type', and 'Type'

# Create the great table
if not df_with_totals.empty:
    tbl = (
    GT(
        df ,
        rowname_col="PO Type",
        groupname_col="Programme"
    )
    .tab_header(
        title="PO Purchase Orders exceeding the Final Date of Implementation"
    )
    # Format "numbers" group as integers (except percentage column)
    .fmt_number(
        columns=[col for col in display_columns if col != '% of Overdue/running grants'],
        # rows=df.index[df["Type"] == "numbers"].tolist(),
        decimals=0,
        use_seps=True
    )
    # Format '% of Overdue/running grants' as percentage with 2 decimal places
    .fmt_percent(
        columns='% of Overdue/running grants',
        # rows=df.index[df["Type"] == "numbers"].tolist(),
        decimals=1,
    )
    .tab_style(
        style.text(color=DARK_BLUE, weight="bold", align="center", font='Arial'),
        locations=loc.header()
    )
 
    .tab_stubhead(label="PO Type")
    .tab_style(
        style=[
            style.text(color=DARK_BLUE, weight="bold", font='Arial', size='medium'),
            style.fill(color=LIGHT_BLUE),
            style.css(f"border-bottom: 2px solid {DARK_BLUE}; border-right: 2px solid {DARK_BLUE}; border-top: 2px solid {DARK_BLUE}; border-left: 2px solid {DARK_BLUE};"),
            style.css("max-width:200px; line-height:1.2"),
        ],
        locations=loc.row_groups()
    )
    .opt_table_font(font="Arial")
    .tab_style(
        style=[
            # style.fill(color=BLUE),
            style.text(color="white", weight="bold", align="center", size='small'),
            style.css("max-width:200px; line-height:1.2")
        ],
        locations=loc.column_labels()
    )
    .tab_style(
        style=[
            # style.fill(color=BLUE),
            style.text(color="white", weight="bold", align="center", size='small'),
            style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2")
        ],
        locations=loc.stubhead()
    )
    .tab_style(
        style=[style.borders(weight="1px", color=DARK_BLUE),
               style.text(size='small')],
        locations=loc.stub()
    )
    .tab_style(
        style=[style.borders(sides="all", color=DARK_BLUE, weight="1px"),
               style.text(align="center", size='small')],
        locations=loc.body()
    )
    .tab_style(
        style=style.borders(color=DARK_BLUE, weight="2px"),
        locations=[loc.column_labels(), loc.stubhead()]
    )
    .tab_style(
        style=[style.fill(color="#D3D3D3"), style.text(color="black", weight="bold")],
        locations=loc.body(rows=df.index[df["PO Type"].str.contains("Total")].tolist())
    )
    .tab_style(
        style=[style.fill(color="#D3D3D3"), style.text(color="black", weight="bold")],
        locations=loc.stub(rows=df.index[df["PO Type"].str.contains("Total")].tolist())
    )
    .tab_options(
        table_body_border_bottom_color=DARK_BLUE,
        table_body_border_bottom_width="2px",
        table_border_right_color=DARK_BLUE,
        table_border_right_width="2px",
        table_border_left_color=DARK_BLUE,
        table_border_left_width="2px",
        table_border_top_color=DARK_BLUE,
        table_border_top_width="2px",
        column_labels_border_top_color=DARK_BLUE,
        column_labels_border_top_width="2px",
        heading_background_color=BLUE,
        row_group_background_color=BLUE

   
    )
    .tab_source_note("Source: Compass")
    .tab_source_note("Reports: Call Overview Report - Budget Follow-Up Report - Ethics Requirements and Issues")
    .tab_style(
        style=[style.text(size="small")],
        locations=loc.footer()
    )
)
else:
    print("No data to display.")

In [13]:
tbl

GT(_tbl_data=   Programme      PO Type  Within 2 Months  Between 2 and 6 Months  \
0      H2020          ADG              0.0                     0.0   
1      H2020          COG              1.0                     1.0   
2      H2020      CSA/SJI              0.0                     0.0   
3      H2020          POC              0.0                     1.0   
4      H2020          STG              0.0                     0.0   
5      H2020          SYG              0.0                     0.0   
6      H2020  Total H2020              1.0                     2.0   
7         HE          ADG              0.0                     0.0   
8         HE          COG              0.0                     0.0   
9         HE      CSA/SJI              0.0                     0.0   
10        HE      Experts              0.0                     0.0   
11        HE          POC              0.0                     0.0   
12        HE          STG              0.0                     0.0   
13        HE          SYG              0.0                     0.0   
14        HE     Total HE              0.0                     0.0   

    More Than 6 Months  Total Overdue  % of Overdue/running grants  \
0                  0.0            0.0                     0.000000   
1                  0.0            2.0                     0.001883   
2                  0.0            0.0                     0.000000   
3                  0.0            1.0                     0.500000   
4                  1.0            1.0                     0.000809   
5                  0.0            0.0                     0.000000   
6                  1.0            4.0                     0.001248   
7                  0.0            0.0                     0.000000   
8                  0.0            0.0                     0.000000   
9                  0.0            0.0                     0.000000   
10                 0.0            0.0                     0.000000   
11                 0.0            0.0                     0.000000   
12                 0.0            0.0                     0.000000   
13                 0.0            0.0                     0.000000   
14                 0.0            0.0                     0.000000   

    Total Commitments with RAL  
0                          805  
1                         1062  
2                            1  
3                            2  
4                         1236  
5                           98  
6                         3204  
7                          726  
8                         1168  
9                            2  
10                          30  
11                         595  
12                        1741  
13                         110  
14                        4372  , _body=<great_tables._gt_data.Body object at 0x0000029409EC4EF0>, _boxhead=Boxhead([ColInfo(var='Programme', type=<ColInfoTypeEnum.row_group: 3>, column_label='Programme', column_align='left', column_width=None), ColInfo(var='PO Type', type=<ColInfoTypeEnum.stub: 2>, column_label='PO Type', column_align='left', column_width=None), ColInfo(var='Within 2 Months', type=<ColInfoTypeEnum.default: 1>, column_label='Within 2 Months', column_align='right', column_width=None), ColInfo(var='Between 2 and 6 Months', type=<ColInfoTypeEnum.default: 1>, column_label='Between 2 and 6 Months', column_align='right', column_width=None), ColInfo(var='More Than 6 Months', type=<ColInfoTypeEnum.default: 1>, column_label='More Than 6 Months', column_align='right', column_width=None), ColInfo(var='Total Overdue', type=<ColInfoTypeEnum.default: 1>, column_label='Total Overdue', column_align='right', column_width=None), ColInfo(var='% of Overdue/running grants', type=<ColInfoTypeEnum.default: 1>, column_label='% of Overdue/running grants', column_align='right', column_width=None), ColInfo(var='Total Commitments with RAL', type=<ColInfoTypeEnum.default: 1>, column_label='Total Commitments with RAL', column_align='right', co

### TTG - TTS 

In [39]:
from __future__ import annotations

import logging, sqlite3, datetime
from pathlib import Path
from typing import List
import math
import numpy as np
import pandas as pd
from great_tables import GT, loc, style, html, md

# our project
from ingestion.db_utils import (
    init_db,                                 # create tables if missing
    fetch_latest_table_data,                 # new version!
    get_alias_last_load,
    get_variable_status,                     # to inspect results
)

from reporting.quarterly_report.utils import RenderContext, BaseModule
from reporting.quarterly_report.report_utils.granting_utils import enrich_grants, _ensure_timedelta_cols, _coerce_date_columns
from ingestion.db_utils import load_report_params
from reporting.quarterly_report.utils import Database, RenderContext

# ─────────────────────────────────────────────────────────────
# 1) open DB – change path if you work on a copy
# ─────────────────────────────────────────────────────────────
DB_PATH = Path("database/reporting.db")
init_db(db_path=DB_PATH)            # no-op if tables already exist

db = Database(str(DB_PATH))         # thin sqlite3 wrapper
conn = db.conn

# ──────────────────────────────────────────────────────────────
# constants – adapt whenever a file-alias changes
# ──────────────────────────────────────────────────────────────
CALL_OVERVIEW_ALIAS   = "call_overview"
BUDGET_FOLLOWUP_ALIAS = "budget_follow_up_report"

EXCLUDE_TOPICS = [
    "ERC-2023-SJI-1", "ERC-2023-SJI",
    "ERC-2024-PERA",
    "HORIZON-ERC-2022-VICECHAIRS-IBA",
    "HORIZON-ERC-2023-VICECHAIRS-IBA",
    "ERC-2025-NCPS-IBA",
    "ERC"
]

MONTHS_ORDER = list(
    pd.date_range("2000-01-01", periods=12, freq="ME").strftime("%B")
)

project_status = ['SIGNED', 'TERMINATED', 'SUSPENDED', 'CLOSED']

cutoff = pd.to_datetime("2025-04-15")

call_overview = fetch_latest_table_data(conn, CALL_OVERVIEW_ALIAS, cutoff)
budget_follow = fetch_latest_table_data(conn, BUDGET_FOLLOWUP_ALIAS, cutoff)

report = 'Quarterly_Report'
db_path = Path("database/reporting.db")

report_params = load_report_params(report_name=report, db_path=db_path)
calls_list = report_params.get("calls_list")
heu_calls_list = report_params.get("HEU_Calls")

# ──────────────────────────────────────────────────────────────
# Define helper functions
# ──────────────────────────────────────────────────────────────
def get_quarter_dates(cutoff: pd.Timestamp, earliest_date: pd.Timestamp) -> tuple[pd.Timestamp, pd.Timestamp]:
    """
    Determine the start date (earliest GA Signature - Commission date) and the end of the quarter
    containing the last full month before the cutoff.
    
    • earliest_date 2024-05-10, cut-off 15-Apr-2025 → 2024-05-10 to 2025-03-31 (Q1 2025)
    • earliest_date 2024-05-10, cut-off 15-Jul-2025 → 2024-05-10 to 2025-06-30 (Q1-Q2 2025)
    • earliest_date 2024-05-10, cut-off 15-Oct-2025 → 2024-05-10 to 2025-09-30 (Q1-Q3 2025)
    • earliest_date 2024-05-10, cut-off 15-Jan-2026 → 2024-05-10 to 2025-12-31 (Q1-Q4 2025)
    """
    # Determine the last full month before the cutoff
    first_day_of_cutoff = cutoff.replace(day=1)
    last_full_month = first_day_of_cutoff - pd.offsets.MonthBegin()
    last_full_month_number = last_full_month.month

    # Use the earliest GA Signature - Commission date as the start
    period_start = earliest_date

    # Determine the end of the quarter based on the last full month
    if last_full_month_number in [1, 2, 3]:  # Q1
        period_end = pd.Timestamp(year=cutoff.year, month=3, day=31)
    elif last_full_month_number in [4, 5, 6]:  # Q2
        period_end = pd.Timestamp(year=cutoff.year, month=6, day=30)
    elif last_full_month_number in [7, 8, 9]:  # Q3
        period_end = pd.Timestamp(year=cutoff.year, month=9, day=30)
    else:  # Q4: [10, 11, 12]
        period_end = pd.Timestamp(year=cutoff.year, month=12, day=31)

    return period_start, period_end

# ────────────────────────────────────────────────────────────
# 2) merge & clean
# ────────────────────────────────────────────────────────────
budget_follow = budget_follow.loc[budget_follow['INVITED '] == 1]

df_grants = (
    call_overview
    .merge(budget_follow, left_on="Grant Number", right_on='Project Number')
    .reset_index()
    .drop_duplicates(subset="Grant Number", keep="last")
    .set_index("Grant Number")
    .sort_index()
)

# Make sure every date column really **is** datetime
_coerce_date_columns(df_grants)



for x in project_status:
    df_grants['GA Signature - Commission'] = np.where(
        (df_grants['GA Signature - Commission'].isnull()) &
        (df_grants['Project Status'] == x) &
        (df_grants['Commitment AO visa'].isnull() == False),
        df_grants['Commitment AO visa'],
        df_grants['GA Signature - Commission']
    )

_ensure_timedelta_cols(df_grants)

df_heu_total = df_grants.loc[
    (df_grants['Call'].isin(heu_calls_list)) & 
    (df_grants['Project Status'].isin(['SIGNED', 'SUSPENDED', 'CLOSED', 'TERMINATED']))
]

HEU_TTG_TOTAL = round(df_heu_total['TTG_timedelta'].mean().total_seconds() / 86400, 2)

df_grants = df_grants[~df_grants["Topic"].isin(EXCLUDE_TOPICS)]
df_grants = df_grants[df_grants["Call"].isin(calls_list)]

# Find the earliest GA Signature - Commission date
earliest_date = df_grants['GA Signature - Commission'].min()
if pd.isna(earliest_date):
    earliest_date = pd.Timestamp(cutoff.year, 1, 1)  # Fallback to start of cutoff year if all dates are NaN
print(f"Earliest GA Signature - Commission date: {earliest_date}")

# Determine the reporting period (from earliest date to the end of the current quarter)
period_start, period_end = get_quarter_dates(cutoff, earliest_date)
print(f"Reporting period for cutoff {cutoff}: {period_start} to {period_end}")

# Filter for signed grants within the reporting period
in_scope = (
    (df_grants['Project Status'].isin(project_status)) &
    (df_grants['GA Signature - Commission'].notna()) &
    (df_grants['GA Signature - Commission'] >= period_start) &
    (df_grants['GA Signature - Commission'] <= period_end)
)

signed = df_grants.loc[in_scope].copy()

HEU_TTG_C_Y = round(signed['TTG_timedelta'].mean().total_seconds() / 86400, 2)

### FOR TOTAL TTS - TTG #######
dfkpi_total = signed.loc[:, ['Call', 'Topic', 'TTG_timedelta', 'TTS_timedelta', 'GA Signature - Commission', 'IS_SIGNED ']]

# Re-formatting KPI columns and KPI average computation
dfkpi = signed.loc[:, ['Call', 'TTG_timedelta', 'TTS_timedelta', 'TTI_timedelta']]
dfkpi.set_index(["Call"], inplace=True, drop=True) 
dfkpi['TTG'] = dfkpi['TTG_timedelta'] / pd.to_timedelta(1, unit='D')
dfkpi['TTS'] = dfkpi['TTS_timedelta'] / pd.to_timedelta(1, unit='D')
dfkpi['TTI'] = dfkpi['TTI_timedelta'] / pd.to_timedelta(1, unit='D')
dfkpi.drop(['TTG_timedelta', 'TTS_timedelta', 'TTI_timedelta'], axis=1, inplace=True)

# AVERAGE IS COMPUTED BY CALL 
df_kpi_prov = dfkpi.groupby(['Call']).mean()
df_kpi_prov.reset_index(inplace=True)

df_g_running = df_grants.loc[df_grants['Project Status'] != 'REJECTED']
df_g_running = df_g_running[['Call']]
df_g_running['Counter'] = 1
df_g_running = df_g_running.groupby('Call').sum()
df_g_running.reset_index(inplace=True)
df_g_running.columns = [
    'Call',
    'Total number of grants excluding rejected'
]

# Step 1: Create a copy with just 'Call' and add a 'Counter' column
df_signed_temp = signed[['Call']].copy()
df_signed_temp['Counter'] = 1

# Step 2: Group by 'Call' and sum 'Counter' to count signed grants
df_signed = df_signed_temp.groupby('Call')['Counter'].sum().reset_index()

# Step 3: Rename columns
df_signed.columns = [
    'Call',
    'Number of Signed Grants'
]

TTS_targets = report_params.get("TTS_Targets")
df_TTS = pd.DataFrame(TTS_targets.items(), columns=["Call", "TTS Target"])

TTG_targets = report_params.get("TTG_Targets")
df_TTG = pd.DataFrame(TTG_targets.items(), columns=["Call", "TTG Target"])

df_Targets = pd.merge(df_TTS, df_TTG, on="Call", how="outer")
df_Targets = df_Targets[df_Targets["Call"].isin(calls_list)]
df_filtered = df_grants.loc[df_grants['Project Status'] != 'REJECTED']

# ********* final table with targets ******* #
merged_df = pd.merge(df_g_running, df_signed, on=["Call"], how="outer")
merged_df = merged_df.fillna(0)
merged_df['Completion Rate'] = merged_df['Number of Signed Grants'] / merged_df['Total number of grants excluding rejected'] 
merged_df = pd.merge(merged_df, df_kpi_prov, on=["Call"], how="outer")

final_df_with_targets = pd.merge(merged_df, df_Targets, on=["Call"], how="outer")
# ********* final table with targets ******* #

def compute_quantiles(call_list: list[str],
                      df_filtered: pd.DataFrame,
                      cutoff: pd.Timestamp,
                      earliest_date: pd.Timestamp
                     ) -> list[pd.DataFrame]:
    """
    Build two data-frames — one for TTS, one for TTG — with 25 % and 50 % percentiles
    for each call in *call_list*.

    Returns
    -------
    [df_tts, df_ttg]
    """
    # ────────────────────────────────────
    # 0. restrict data set + pre-processing
    # ────────────────────────────────────
    df_filtered = df_filtered[df_filtered['Call'].isin(call_list)].copy()
    df_filtered['GA Signature - Commission'] = pd.to_datetime(
        df_filtered['GA Signature - Commission'], errors='coerce')

    # signed periods
    period_start, period_end = get_quarter_dates(cutoff, earliest_date)

    signed_statuses = ['SIGNED', 'SUSPENDED', 'CLOSED', 'TERMINATED']

    # derive ACTIVE / SIGNED flags
    df_filtered['ACTIVE'] = (df_filtered['Project Status'] == 'UNDER_PREPARATION').astype(int)

    df_filtered['SIGNED'] = (
        (df_filtered['Project Status'].isin(signed_statuses)) &
        (df_filtered['GA Signature - Commission'].notna()) &
        (df_filtered['GA Signature - Commission'] >= period_start) &
        (df_filtered['GA Signature - Commission'] <= period_end)
    ).astype(int)

    # keep a grant ACTIVE if it is not signed in the current period
    df_filtered.loc[df_filtered['SIGNED'] == 0, 'ACTIVE'] = 1

    # ────────────────────────────────────
    # 1. loop over each call and collect KPI values
    # ────────────────────────────────────
    results = {
        'Call': [],
        'Total number of grants excluding rejected': [],
        'Total number of signed grants': [],
        'TTS_25': [], 'TTS_50': [],
        'TTG_25': [], 'TTG_50': []
    }

    for call in call_list:
        kpi = df_filtered[df_filtered['Call'] == call].copy()

        if kpi.empty:
            # still add an empty row so the output keeps all calls
            for key in ['Call',
                        'Total number of grants excluding rejected',
                        'Total number of signed grants',
                        'TTS_25', 'TTS_50', 'TTG_25', 'TTG_50']:
                results[key].append(np.nan if key != 'Call' else call)
            continue

        # sort so grants with TTG == 0 land at the bottom
        kpi['Class'] = np.where(kpi['TTG_timedelta'] != pd.Timedelta(0), 'A', 'B')
        kpi = kpi.sort_values(['Class', 'TTG_timedelta'])

        active_n = kpi['ACTIVE'].sum()
        signed_n = kpi['SIGNED'].sum()

        def _percentile(series: pd.Series, pct: float):
            """Return value at *pct* (0–1) position of *series* after prior sort."""
            if len(series) == 0:
                return np.nan
            idx = int(np.floor(len(series) * pct))
            return series.iloc[idx]

        tts_25 = _percentile(kpi['TTS_timedelta'], 0.25) / pd.Timedelta('1D')
        tts_50 = _percentile(kpi['TTS_timedelta'], 0.50) / pd.Timedelta('1D')
        ttg_25 = _percentile(kpi['TTG_timedelta'], 0.25) / pd.Timedelta('1D')
        ttg_50 = _percentile(kpi['TTG_timedelta'], 0.50) / pd.Timedelta('1D')

        # write to results
        results['Call'].append(call)
        results['Total number of grants excluding rejected'].append(active_n + signed_n)
        results['Total number of signed grants'].append(signed_n)
        results['TTS_25'].append(tts_25)
        results['TTS_50'].append(tts_50)
        results['TTG_25'].append(ttg_25)
        results['TTG_50'].append(ttg_50)

    # ────────────────────────────────────
    # 2. build the two output tables
    # ────────────────────────────────────
    df_tts = pd.DataFrame({
        'Call': results['Call'],
        'Total number of grants excluding rejected':
            results['Total number of grants excluding rejected'],
        'Total Number of Signed Grants':
            results['Total number of signed grants'],
        'First 25% (days)': results['TTS_25'],
        'First 50% (days)': results['TTS_50']
    })

    df_ttg = pd.DataFrame({
        'Call': results['Call'],
        'Total number of grants excluding rejected':
            results['Total number of grants excluding rejected'],
        'Total Number of Signed Grants':
            results['Total number of signed grants'],
        'First 25% (days)': results['TTG_25'],
        'First 50% (days)': results['TTG_50']
    })

    # optional pretty-print formatting (leave numeric so caller can still use values)
    for df in (df_tts, df_ttg):
        for col in ['Total number of grants excluding rejected',
                    'Total Number of Signed Grants']:
            df[col] = df[col].astype('Int64')  # keeps NaNs while showing as ints

    return [df_tts, df_ttg]


# ********* final tables with quantiles ******* #
[df_tts, df_ttg] = compute_quantiles(calls_list, df_filtered, cutoff, earliest_date )

def transpose_table (df):
    # Set 'Call' as index temporarily for pivoting
    df_pivot = df.set_index('Call')
    # Select and rename rows
    df_transposed = pd.DataFrame({
        'Total number of grants excluding rejected': df_pivot['Total number of grants excluding rejected'],
        'Total number of Signed Grants': df_pivot['Total Number of Signed Grants'],
        'First 25%': df_pivot['First 25% (days)'],
        'First 50%': df_pivot['First 50% (days)']
    }).T

    return df_transposed



q_tts =  transpose_table (df_tts)
q_ttg =  transpose_table (df_ttg)


def transpose_table(df, metric: str):
    """
    Transposes selected metrics for TTG or TTS based on 'Call'.

    Parameters:
        df (pd.DataFrame): Input dataframe with columns including 'Call', 
                           'Number of Signed Grants', '<metric>', 
                           '<metric> Target', and 'Completion Rate'.
        metric (str): Either 'TTG' or 'TTS'.

    Returns:
        pd.DataFrame: Transposed dataframe with renamed rows.
    """
    if metric not in ['TTG', 'TTS']:
        raise ValueError("Metric must be 'TTG' or 'TTS'")

    expected_cols = {
        'Call', 
        'Number of Signed Grants', 
        metric, 
        f'{metric} Target', 
        'Completion Rate'
    }

    missing = expected_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns in input DataFrame: {missing}")

    df_pivot = df.set_index('Call')
    df_transposed = pd.DataFrame({
        'Total number of Signed Grants': df_pivot['Number of Signed Grants'],
        f'Average {metric}': df_pivot[metric],
        'Target': df_pivot[f'{metric} Target'],
        'Completion Rate': df_pivot['Completion Rate']
    }).T

    return df_transposed

df_TTG = transpose_table(final_df_with_targets, 'TTG')
df_TTS = transpose_table(final_df_with_targets, 'TTS')


final_df_overview = pd.merge(final_df_with_targets, df_tts, on='Call',how="outer"  )
final_df_tts_overview = final_df_overview[['Call','First 25% (days)', 'First 50% (days)', 'TTS', 'Completion Rate']]


DEBUG:root:Fetching latest data for table_alias: call_overview, cutoff: 2025-04-15T00:00:00
DEBUG:root:Upload log query results for call_overview: [('2025-04-18T19:59:00.734594', 1), ('2025-05-08T18:12:23.673792', 8)]
DEBUG:root:Checking upload_id: 1, uploaded_at: 2025-04-18T19:59:00.734594
DEBUG:root:No data found for upload_id 1 in call_overview
DEBUG:root:Checking upload_id: 8, uploaded_at: 2025-05-08T18:12:23.673792
DEBUG:root:Fetched 13295 rows from call_overview with upload_id 8
DEBUG:root:Fetching latest data for table_alias: budget_follow_up_report, cutoff: 2025-04-15T00:00:00
DEBUG:root:Upload log query results for budget_follow_up_report: [('2025-05-08T18:15:43.583290', 9)]
DEBUG:root:Checking upload_id: 9, uploaded_at: 2025-05-08T18:15:43.583290
DEBUG:root:Fetched 16470 rows from budget_follow_up_report with upload_id 9


Earliest GA Signature - Commission date: 2024-07-10 00:00:00
Reporting period for cutoff 2025-04-15 00:00:00: 2024-07-10 00:00:00 to 2025-03-31 00:00:00


In [385]:

q_tts


Call,ERC-2024-POC (03/24),ERC-2024-STG,ERC-2024-COG,ERC-2024-SyG,ERC-2024-POC (09/24)
Total number of grants excluding rejected,111,493,329,57,134
Total number of Signed Grants,109,471,210,40,85
First 25%,41.0,72.0,90.0,117.0,43.0
First 50%,69.0,97.0,111.0,132.0,63.0


In [386]:
q_ttg

Call,ERC-2024-POC (03/24),ERC-2024-STG,ERC-2024-COG,ERC-2024-SyG,ERC-2024-POC (09/24)
Total number of grants excluding rejected,111,493,329,57,134
Total number of Signed Grants,109,471,210,40,85
First 25%,147.0,323.0,433.0,452.0,157.0
First 50%,175.0,348.0,454.0,467.0,177.0


In [387]:

df_TTS 

Call,ERC-2024-COG,ERC-2024-POC (03/24),ERC-2024-POC (09/24),ERC-2024-STG,ERC-2024-SyG
Total number of Signed Grants,210,109,85,471,40
Average TTS,92.738095,70.0,50.294118,103.643312,114.05
Target,120,120,120,120,140
Completion Rate,0.638298,0.981982,0.634328,0.955375,0.701754


In [388]:
df_tts

,Call,Total number of grants excluding rejected,Total Number of Signed Grants,First 25% (days),First 50% (days)
0,ERC-2024-POC (03/24),111,109,41.0,69.0
1,ERC-2024-STG,493,471,72.0,97.0
2,ERC-2024-COG,329,210,90.0,111.0
3,ERC-2024-SyG,57,40,117.0,132.0
4,ERC-2024-POC (09/24),134,85,43.0,63.0


In [389]:

final_df_tts_overview

,Call,First 25% (days),First 50% (days),TTS,Completion Rate
0,ERC-2024-COG,90.0,111.0,92.738095,0.638298
1,ERC-2024-POC (03/24),41.0,69.0,70.000000,0.981982
2,ERC-2024-POC (09/24),43.0,63.0,50.294118,0.634328
3,ERC-2024-STG,72.0,97.0,103.643312,0.955375
4,ERC-2024-SyG,117.0,132.0,114.050000,0.701754


In [2]:
table_colors = report_params.get("TABLE_COLORS")
table_colors
# Unpack the ones you need
BLUE = table_colors['BLUE']
LIGHT_BLUE = table_colors['LIGHT_BLUE']
DARK_BLUE = table_colors['DARK_BLUE']


In [3]:
df_TTG.reset_index(inplace=True)
# df_TTG = df_TTG.drop(columns=['level_0'])

In [392]:
df_TTG

Call,index,ERC-2024-COG,ERC-2024-POC (03/24),ERC-2024-POC (09/24),ERC-2024-STG,ERC-2024-SyG
0,Total number of Signed Grants,210,109,85,471,40
1,Average TTG,435.738095,176.0,164.294118,354.649682,449.05
2,Target,429,226,226,420,511
3,Completion Rate,0.638298,0.981982,0.634328,0.955375,0.701754


In [4]:
# Build GreatTables object
try:
    tbl_ttg = (
        GT(
            df_TTG,
            rowname_col="index"
        )
     
       .tab_header(
            title=html(
                f"<strong style='color: {BLUE};'>TIME TO GRANT</strong>  "
                f"<span style='color: {BLUE}; font-style: italic; font-size: smaller;'>(Main list & Reserve list)</span>"
            )
        )

    .tab_stubhead(
               label=html(f"<span style='color: white ; font-size: large; align-text: center; margin-left: 5px; margin-bottom: 80px;' >Call</span>")
               )
        
        # GENERAL FORMATTING
        # Table Outline
        .opt_table_outline(style = "solid", width = "3px", color =  DARK_BLUE) 
        # Arial font
        .opt_table_font(font="Arial")

        # Header and stub styling
    
        .tab_style(
            style=[
                style.fill(color=BLUE),
                style.text(color="white", weight="bold", align='center'),
                style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;")
            ],
            locations=loc.column_labels()
         )

   
        # # Table borders
        .tab_options(table_body_border_bottom_color=DARK_BLUE, table_body_border_bottom_width="2px")
        .tab_options(table_body_border_top_color=DARK_BLUE, table_body_border_top_width="2px")
        # .tab_options(table_border_right_color=DARK_BLUE, table_border_right_width="2px")
        # .tab_options(table_border_left_color=DARK_BLUE, table_border_left_width="2px")
        # .tab_options(table_border_top_color=DARK_BLUE, table_border_top_width="2px")
        .tab_options(column_labels_border_top_color=DARK_BLUE, column_labels_border_top_width="2px")
        .tab_options(column_labels_border_bottom_color=DARK_BLUE, column_labels_border_bottom_width="2px")
        # .tab_options(row_group_border_top_color=DARK_BLUE, row_group_border_top_width="2px")
        # .tab_options( stub_row_group_border_color=DARK_BLUE, stub_row_group_border_width="2px")
        # .tab_options(heading_background_color= 'white')
        .tab_options( column_labels_background_color = BLUE)
        # .tab_options( row_striping_background_color = DARK_BLUE)
        # .tab_options( table_body_vlines_color= DARK_BLUE, table_body_vlines_width="2px")
         .tab_options( row_group_background_color = 'red')


        # BODY
        .fmt_percent(
            rows=["Completion Rate"],  # or use `where` with a condition
            decimals=1
        )

        .fmt_number(
            rows=["Average TTG"],
            decimals=1,
            accounting=False
        )

        # Source notes
        .tab_source_note("Source: Compass")
        .tab_source_note("Reports: Budgetary Execution Details - Call Overview Report")
    )
except Exception as e:
            logging.error(f"Error building GreatTables object: {str(e)}")
            # Return the aggregated DataFrame without styling if table creation fails
           

In [5]:
tbl_ttg

GT(_tbl_data=Call                          index ERC-2024-COG ERC-2024-POC (03/24)  \
0     Total number of Signed Grants          210                  109   
1                       Average TTG   435.738095                176.0   
2                            Target          429                  226   
3                   Completion Rate     0.638298             0.981982   

Call ERC-2024-POC (09/24) ERC-2024-STG ERC-2024-SyG  
0                      85          471           40  
1              164.294118   354.649682       449.05  
2                     226          420          511  
3                0.634328     0.955375     0.701754  , _body=<great_tables._gt_data.Body object at 0x0000028725CF4D70>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='left', column_width=None), ColInfo(var='ERC-2024-COG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-COG', column_align='right', column_width=None), ColInfo(var='ERC-2024-POC (03/24)', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-POC (03/24)', column_align='right', column_width=None), ColInfo(var='ERC-2024-POC (09/24)', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-POC (09/24)', column_align='right', column_width=None), ColInfo(var='ERC-2024-STG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-STG', column_align='right', column_width=None), ColInfo(var='ERC-2024-SyG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-SyG', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000028725CF4AD0>, _spanners=Spanners([]), _heading=Heading(title=Html(text="<strong style='color: #004A99;'>TIME TO GRANT</strong>  <span style='color: #004A99; font-style: italic; font-size: smaller;'>(Main list & Reserve list)</span>"), subtitle=None, preheader=None), _stubhead=Html(text="<span style='color: white ; font-size: large; align-text: center; margin-left: 5px; margin-bottom: 80px;' >Call</span>"), _source_notes=['Source: Compass', 'Reports: Budgetary Execution Details - Call Overview Report'], _footnotes=[], _styles=[StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-COG', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-POC (03/24)', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-POC (09/24)', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-STG', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, c

In [421]:
df_TTS.reset_index(inplace=True)

In [ ]:
# Build GreatTables object
try:
    tbl_tts = (
        GT(
            df_TTS,
            rowname_col="index"
        )
     
       .tab_header(
            title=html(
                f"<strong style='color: {BLUE};'>TIME TO SIGN</strong>  "
                f"<span style='color: {BLUE}; font-style: italic; font-size: smaller;'>(Main list & Reserve list)</span>"
            )
        )

    .tab_stubhead(
               label=html(f"<span style='color: white ; font-size: large; align-text: center; margin-left: 5px; margin-bottom: 80px;' >Call</span>")
               )
        
        # GENERAL FORMATTING
        # Table Outline
        .opt_table_outline(style = "solid", width = "3px", color =  DARK_BLUE) 
        # Arial font
        .opt_table_font(font="Arial")

        # Header and stub styling
    
        .tab_style(
            style=[
                style.fill(color=BLUE),
                style.text(color="white", weight="bold", align='center'),
                style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;")
            ],
            locations=loc.column_labels()
         )

   
        # # Table borders
        .tab_options(table_body_border_bottom_color=DARK_BLUE, table_body_border_bottom_width="2px")
        .tab_options(table_body_border_top_color=DARK_BLUE, table_body_border_top_width="2px")
        # .tab_options(table_border_right_color=DARK_BLUE, table_border_right_width="2px")
        # .tab_options(table_border_left_color=DARK_BLUE, table_border_left_width="2px")
        # .tab_options(table_border_top_color=DARK_BLUE, table_border_top_width="2px")
        .tab_options(column_labels_border_top_color=DARK_BLUE, column_labels_border_top_width="2px")
        .tab_options(column_labels_border_bottom_color=DARK_BLUE, column_labels_border_bottom_width="2px")
        # .tab_options(row_group_border_top_color=DARK_BLUE, row_group_border_top_width="2px")
        # .tab_options( stub_row_group_border_color=DARK_BLUE, stub_row_group_border_width="2px")
        # .tab_options(heading_background_color= 'white')
        .tab_options( column_labels_background_color = BLUE)
        # .tab_options( row_striping_background_color = DARK_BLUE)
        # .tab_options( table_body_vlines_color= DARK_BLUE, table_body_vlines_width="2px")
         .tab_options( row_group_background_color = 'red')


        # BODY
        .fmt_percent(
            rows=["Completion Rate"],  # or use `where` with a condition
            decimals=1
        )

        .fmt_number(
            rows=["Average TTS"],
            decimals=1,
            accounting=False
        )

        # Source notes
        .tab_source_note("Source: Compass")
        .tab_source_note("Reports: Budgetary Execution Details - Call Overview Report")
    )
except Exception as e:
            logging.error(f"Error building GreatTables object: {str(e)}")
            # Return the aggregated DataFrame without styling if table creation fails
           

In [427]:
tbl_tts

GT(_tbl_data=Call                          index ERC-2024-COG ERC-2024-POC (03/24)  \
0     Total number of Signed Grants          210                  109   
1                       Average TTS    92.738095                 70.0   
2                            Target          120                  120   
3                   Completion Rate     0.638298             0.981982   

Call ERC-2024-POC (09/24) ERC-2024-STG ERC-2024-SyG  
0                      85          471           40  
1               50.294118   103.643312       114.05  
2                     120          120          140  
3                0.634328     0.955375     0.701754  , _body=<great_tables._gt_data.Body object at 0x0000023C057F45F0>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='left', column_width=None), ColInfo(var='ERC-2024-COG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-COG', column_align='right', column_width=None), ColInfo(var='ERC-2024-POC (03/24)', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-POC (03/24)', column_align='right', column_width=None), ColInfo(var='ERC-2024-POC (09/24)', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-POC (09/24)', column_align='right', column_width=None), ColInfo(var='ERC-2024-STG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-STG', column_align='right', column_width=None), ColInfo(var='ERC-2024-SyG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-SyG', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000023C2F3E87D0>, _spanners=Spanners([]), _heading=Heading(title=Html(text="<strong style='color: #004A99;'>TIME TO SIGN</strong>  <span style='color: #004A99; font-style: italic; font-size: smaller;'>(Main list & Reserve list)</span>"), subtitle=None, preheader=None), _stubhead=Html(text="<span style='color: white ; font-size: large; align-text: center; margin-left: 60px; margin-bottom: 80px;' >Call</span>"), _source_notes=['Source: Compass', 'Reports: Budgetary Execution Details - Call Overview Report'], _footnotes=[], _styles=[StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-COG', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-POC (03/24)', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-POC (09/24)', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-STG', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, c

In [435]:
q_ttg.reset_index(inplace=True)

In [436]:
q_ttg

Call,index,ERC-2024-POC (03/24),ERC-2024-STG,ERC-2024-COG,ERC-2024-SyG,ERC-2024-POC (09/24)
0,Total number of grants excluding rejected,111,493,329,57,134
1,Total number of Signed Grants,109,471,210,40,85
2,First 25%,147.0,323.0,433.0,452.0,157.0
3,First 50%,175.0,348.0,454.0,467.0,177.0


In [ ]:
# Build GreatTables object
try:
    tbl_q_ttg = (
        GT(
           q_ttg,
            rowname_col="index"
        )
     
       .tab_header(
            title=html(
                f"<strong style='color: {BLUE};'>TIME TO GRANT - Quantiles </strong>  "
                #  f"<span style='color: {BLUE}; font-style: italic; font-size: smaller;'>With Quantiles</span>"
                 f"<span style='color: {BLUE}; font-style: italic; font-size: smaller;'>(Main list & Reserve list)</span>"
            )
        )

    .tab_stubhead(
               label=html(f"<span style='color: white ; font-size: large; align-text: center; margin-left: 5px; margin-bottom: 80px;' >Call</span>")
               )
        
        # GENERAL FORMATTING
        # Table Outline
        .opt_table_outline(style = "solid", width = "3px", color =  DARK_BLUE) 
        # Arial font
        .opt_table_font(font="Arial")

        # Header and stub styling
    
        .tab_style(
            style=[
                style.fill(color=BLUE),
                style.text(color="white", weight="bold", align='center'),
                style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;")
            ],
            locations=loc.column_labels()
         )

   
        # # Table borders
        .tab_options(table_body_border_bottom_color=DARK_BLUE, table_body_border_bottom_width="2px")
        .tab_options(table_body_border_top_color=DARK_BLUE, table_body_border_top_width="2px")
        # .tab_options(table_border_right_color=DARK_BLUE, table_border_right_width="2px")
        # .tab_options(table_border_left_color=DARK_BLUE, table_border_left_width="2px")
        # .tab_options(table_border_top_color=DARK_BLUE, table_border_top_width="2px")
        .tab_options(column_labels_border_top_color=DARK_BLUE, column_labels_border_top_width="2px")
        .tab_options(column_labels_border_bottom_color=DARK_BLUE, column_labels_border_bottom_width="2px")
        # .tab_options(row_group_border_top_color=DARK_BLUE, row_group_border_top_width="2px")
        # .tab_options( stub_row_group_border_color=DARK_BLUE, stub_row_group_border_width="2px")
        # .tab_options(heading_background_color= 'white')
        .tab_options( column_labels_background_color = BLUE)
        # .tab_options( row_striping_background_color = DARK_BLUE)
        # .tab_options( table_body_vlines_color= DARK_BLUE, table_body_vlines_width="2px")
         .tab_options( row_group_background_color = 'red')


        # BODY
        .fmt_percent(
            rows=["Completion Rate"],  # or use `where` with a condition
            decimals=1
        )

        # .fmt_number(
        #     rows=["Average TTS"],
        #     decimals=1,
        #     accounting=False
        # )

        # Source notes
        .tab_source_note("Source: Compass")
        .tab_source_note("Reports: Budgetary Execution Details - Call Overview Report")
    )
except Exception as e:
            logging.error(f"Error building GreatTables object: {str(e)}")
            # Return the aggregated DataFrame without styling if table creation fails
           

In [442]:
tbl_q_ttg 

GT(_tbl_data=Call                                      index ERC-2024-POC (03/24)  \
0     Total number of grants excluding rejected                  111   
1                 Total number of Signed Grants                  109   
2                                     First 25%                147.0   
3                                     First 50%                175.0   

Call ERC-2024-STG ERC-2024-COG ERC-2024-SyG ERC-2024-POC (09/24)  
0             493          329           57                  134  
1             471          210           40                   85  
2           323.0        433.0        452.0                157.0  
3           348.0        454.0        467.0                177.0  , _body=<great_tables._gt_data.Body object at 0x0000023C31D77FC0>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='left', column_width=None), ColInfo(var='ERC-2024-POC (03/24)', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-POC (03/24)', column_align='right', column_width=None), ColInfo(var='ERC-2024-STG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-STG', column_align='right', column_width=None), ColInfo(var='ERC-2024-COG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-COG', column_align='right', column_width=None), ColInfo(var='ERC-2024-SyG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-SyG', column_align='right', column_width=None), ColInfo(var='ERC-2024-POC (09/24)', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-POC (09/24)', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000023C3268DBB0>, _spanners=Spanners([]), _heading=Heading(title=Html(text="<strong style='color: #004A99;'>TIME TO GRANT - Quantiles </strong>  <span style='color: #004A99; font-style: italic; font-size: smaller;'>(Main list & Reserve list)</span>"), subtitle=None, preheader=None), _stubhead=Html(text="<span style='color: white ; font-size: large; align-text: center; margin-left: 60px; margin-bottom: 80px;' >Call</span>"), _source_notes=['Source: Compass', 'Reports: Budgetary Execution Details - Call Overview Report'], _footnotes=[], _styles=[StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-POC (03/24)', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-STG', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-COG', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-SyG', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')])

In [443]:
q_tts.reset_index(inplace=True)

In [ ]:
# Build GreatTables object
try:
    tbl_q_tts = (
        GT(
           q_tts,
            rowname_col="index"
        )
     
       .tab_header(
            title=html(
                f"<strong style='color: {BLUE};'>TIME TO SIGN - Quantiles </strong>  "
                #  f"<span style='color: {BLUE}; font-style: italic; font-size: smaller;'>With Quantiles</span>"
                 f"<span style='color: {BLUE}; font-style: italic; font-size: smaller;'>(Main list & Reserve list)</span>"
            )
        )

    .tab_stubhead(
               label=html(f"<span style='color: white ; font-size: large; align-text: center; margin-left: 5px; margin-bottom: 80px;' >Call</span>")
               )
        
        # GENERAL FORMATTING
        # Table Outline
        .opt_table_outline(style = "solid", width = "3px", color =  DARK_BLUE) 
        # Arial font
        .opt_table_font(font="Arial")

        # Header and stub styling
    
        .tab_style(
            style=[
                style.fill(color=BLUE),
                style.text(color="white", weight="bold", align='center'),
                style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;")
            ],
            locations=loc.column_labels()
         )

   
        # # Table borders
        .tab_options(table_body_border_bottom_color=DARK_BLUE, table_body_border_bottom_width="2px")
        .tab_options(table_body_border_top_color=DARK_BLUE, table_body_border_top_width="2px")
        # .tab_options(table_border_right_color=DARK_BLUE, table_border_right_width="2px")
        # .tab_options(table_border_left_color=DARK_BLUE, table_border_left_width="2px")
        # .tab_options(table_border_top_color=DARK_BLUE, table_border_top_width="2px")
        .tab_options(column_labels_border_top_color=DARK_BLUE, column_labels_border_top_width="2px")
        .tab_options(column_labels_border_bottom_color=DARK_BLUE, column_labels_border_bottom_width="2px")
        # .tab_options(row_group_border_top_color=DARK_BLUE, row_group_border_top_width="2px")
        # .tab_options( stub_row_group_border_color=DARK_BLUE, stub_row_group_border_width="2px")
        # .tab_options(heading_background_color= 'white')
        .tab_options( column_labels_background_color = BLUE)
        # .tab_options( row_striping_background_color = DARK_BLUE)
        # .tab_options( table_body_vlines_color= DARK_BLUE, table_body_vlines_width="2px")
         .tab_options( row_group_background_color = 'red')


        # BODY
        .fmt_percent(
            rows=["Completion Rate"],  # or use `where` with a condition
            decimals=1
        )

        # .fmt_number(
        #     rows=["Average TTS"],
        #     decimals=1,
        #     accounting=False
        # )

        # Source notes
        .tab_source_note("Source: Compass")
        .tab_source_note("Reports: Budgetary Execution Details - Call Overview Report")
    )
except Exception as e:
            logging.error(f"Error building GreatTables object: {str(e)}")
            # Return the aggregated DataFrame without styling if table creation fails

In [446]:
tbl_q_tts

GT(_tbl_data=Call                                      index ERC-2024-POC (03/24)  \
0     Total number of grants excluding rejected                  111   
1                 Total number of Signed Grants                  109   
2                                     First 25%                 41.0   
3                                     First 50%                 69.0   

Call ERC-2024-STG ERC-2024-COG ERC-2024-SyG ERC-2024-POC (09/24)  
0             493          329           57                  134  
1             471          210           40                   85  
2            72.0         90.0        117.0                 43.0  
3            97.0        111.0        132.0                 63.0  , _body=<great_tables._gt_data.Body object at 0x0000023C070B34D0>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='left', column_width=None), ColInfo(var='ERC-2024-POC (03/24)', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-POC (03/24)', column_align='right', column_width=None), ColInfo(var='ERC-2024-STG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-STG', column_align='right', column_width=None), ColInfo(var='ERC-2024-COG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-COG', column_align='right', column_width=None), ColInfo(var='ERC-2024-SyG', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-SyG', column_align='right', column_width=None), ColInfo(var='ERC-2024-POC (09/24)', type=<ColInfoTypeEnum.default: 1>, column_label='ERC-2024-POC (09/24)', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000023C33608AD0>, _spanners=Spanners([]), _heading=Heading(title=Html(text="<strong style='color: #004A99;'>TIME TO SIGN - Quantiles </strong>  <span style='color: #004A99; font-style: italic; font-size: smaller;'>(Main list & Reserve list)</span>"), subtitle=None, preheader=None), _stubhead=Html(text="<span style='color: white ; font-size: large; align-text: center; margin-left: 60px; margin-bottom: 80px;' >Call</span>"), _source_notes=['Source: Compass', 'Reports: Budgetary Execution Details - Call Overview Report'], _footnotes=[], _styles=[StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-POC (03/24)', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-STG', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-COG', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='ERC-2024-SyG', rownum=None, colnum=None, styles=[CellStyleFill(color='#004A99'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]),

In [6]:
final_df_tts_overview = final_df_tts_overview.rename(columns={"Completion Rate": "Completion_Rate"})
final_df_tts_overview 

,Call,First 25% (days),First 50% (days),TTS,Completion_Rate
0,ERC-2024-COG,90.0,111.0,92.738095,0.638298
1,ERC-2024-POC (03/24),41.0,69.0,70.000000,0.981982
2,ERC-2024-POC (09/24),43.0,63.0,50.294118,0.634328
3,ERC-2024-STG,72.0,97.0,103.643312,0.955375
4,ERC-2024-SyG,117.0,132.0,114.050000,0.701754


In [9]:
# Build GreatTables object
try:
    tbl_grants_tts_overview = (
        GT(
           final_df_tts_overview,
            rowname_col="Call"
        )
     
       .tab_header(
           title=html(
        f"<strong style=' font-size: medium; text-align: left; display: block;'>Time-to-Sign HEU</strong>"
            )
        )

.tab_stubhead(
    label=html(
        f"<span style=' font-size: smaller; text-align: left; display: block; white-space: normal; max-width: 400px;'>"
        "Time to Sign: From the information letter<br>sent to the signature of the Grant Agreement"
        "</span>"
    )
)
        # GENERAL FORMATTING
        # Table Outline
        .opt_table_outline(style = "solid", width = "3px", color =  DARK_BLUE) 
        # Arial font
        .opt_table_font(font="Arial")

        # Header and stub styling
    
        .tab_style(
            style=[
                style.fill(color=LIGHT_BLUE),
                style.text(color="white", weight="bold", align='center'),
                style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;")
            ],
            locations=loc.column_labels()
         )

        .cols_label(
    Completion_Rate=html(
        "Completion Rate <span style='font-size: smaller;'> (8)</span> <br> --------------------------</br> "
        "<span style='font-size: smaller;'>(Main + Reserve lists)</span>"
    )   
)

   
        # # Table borders
        .tab_options(table_body_border_bottom_color=DARK_BLUE, table_body_border_bottom_width="2px")
        .tab_options(table_body_border_top_color=DARK_BLUE, table_body_border_top_width="2px")
        # .tab_options(table_border_right_color=DARK_BLUE, table_border_right_width="2px")
        # .tab_options(table_border_left_color=DARK_BLUE, table_border_left_width="2px")
        # .tab_options(table_border_top_color=DARK_BLUE, table_border_top_width="2px")
        .tab_options(column_labels_border_top_color=DARK_BLUE, column_labels_border_top_width="2px")
        .tab_options(column_labels_border_bottom_color=DARK_BLUE, column_labels_border_bottom_width="2px")
        # .tab_options(row_group_border_top_color=DARK_BLUE, row_group_border_top_width="2px")
        # .tab_options( stub_row_group_border_color=DARK_BLUE, stub_row_group_border_width="2px")
        # .tab_options(heading_background_color= 'white')
        .tab_options( column_labels_background_color = LIGHT_BLUE)
        # .tab_options( row_striping_background_color = DARK_BLUE)
        # .tab_options( table_body_vlines_color= DARK_BLUE, table_body_vlines_width="2px")
         .tab_options( row_group_background_color = 'red')


        # BODY
        .fmt_percent(
            columns=["Completion_Rate"],  # or use `where` with a condition
            decimals=1
        )

        .fmt_number(
            columns=["TTS"],
            decimals=1,
            accounting=False
        )

        # # Source notes
        # .tab_source_note("Source: Compass")
        # .tab_source_note("Reports: Budgetary Execution Details - Call Overview Report")
    )
except Exception as e:
            logging.error(f"Error building GreatTables object: {str(e)}")
            # Return the aggregated DataFrame without styling if table creation fails

In [10]:
tbl_grants_tts_overview

GT(_tbl_data=                   Call  First 25% (days)  First 50% (days)         TTS  \
0          ERC-2024-COG              90.0             111.0   92.738095   
1  ERC-2024-POC (03/24)              41.0              69.0   70.000000   
2  ERC-2024-POC (09/24)              43.0              63.0   50.294118   
3          ERC-2024-STG              72.0              97.0  103.643312   
4          ERC-2024-SyG             117.0             132.0  114.050000   

   Completion_Rate  
0         0.638298  
1         0.981982  
2         0.634328  
3         0.955375  
4         0.701754  , _body=<great_tables._gt_data.Body object at 0x0000028725053460>, _boxhead=Boxhead([ColInfo(var='Call', type=<ColInfoTypeEnum.stub: 2>, column_label='Call', column_align='left', column_width=None), ColInfo(var='First 25% (days)', type=<ColInfoTypeEnum.default: 1>, column_label='First 25% (days)', column_align='right', column_width=None), ColInfo(var='First 50% (days)', type=<ColInfoTypeEnum.default: 1>, column_label='First 50% (days)', column_align='right', column_width=None), ColInfo(var='TTS', type=<ColInfoTypeEnum.default: 1>, column_label='TTS', column_align='right', column_width=None), ColInfo(var='Completion_Rate', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="Completion Rate <span style='font-size: smaller;'> (8)</span> <br> --------------------------</br> <span style='font-size: smaller;'>(Main + Reserve lists)</span>"), column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000028727C28CD0>, _spanners=Spanners([]), _heading=Heading(title=Html(text="<strong style=' font-size: medium; text-align: left; display: block;'>Time-to-Sign HEU</strong>"), subtitle=None, preheader=None), _stubhead=Html(text="<span style=' font-size: smaller; text-align: left; display: block; white-space: normal; max-width: 400px;'>Time to Sign: From the information letter<br>sent to the signature of the Grant Agreement</span>"), _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='First 25% (days)', rownum=None, colnum=None, styles=[CellStyleFill(color='#d6e6f4'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='First 50% (days)', rownum=None, colnum=None, styles=[CellStyleFill(color='#d6e6f4'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='TTS', rownum=None, colnum=None, styles=[CellStyleFill(color='#d6e6f4'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='Completion_Rate', rownum=None, colnum=None, styles=[CellStyleFill(color='#d6e6f4'), CellStyleText(color='white', font=None, size=None, align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')])], _locale=<great_tables._gt_data.Locale object at 0x0000028727C28F50>, _formats=[<great_tables._gt_data.FormatInfo object at 0x00000287250A7BB0>, <great_tables._gt_data.FormatInfo object at 0x0000028725

In [30]:
df_heu_total.head(2)


,index,Call,Acronym,Project Title,Instrument,Topic,Coordinator,Duration,Nr of periods,Eu contribution,...,IN_BUDGET_PLANNER,IN_ABAC,IS_SIGNED,GAP_STEP,PREFIN_STEP,upload_id_y,uploaded_at_y,TTG_timedelta,TTS_timedelta,TTI_timedelta
Grant Number,,,,,,,,,,,,,,,,,,,,,
101039048,8857,ERC-2021-STG,GanESS,Gaseous detectors for neutrino physics at the ...,HORIZON-ERC,ERC-2021-STG,FUNDACION DONOSTIA INTERNATIONAL PHYSICS CENTER,72.0,2.0,1496205.0,...,1,0,1,None,None,9,2025-05-08T18:15:33.117272,288 days,36 days,252 days
101039060,8858,ERC-2021-STG,PalaeOrigins,Tracing the Epipalaeolithic origins of plant m...,HORIZON-ERC,ERC-2021-STG,UNIVERSIDAD DEL PAIS VASCO/ EUSKAL HERRIKO UNI...,60.0,2.0,1499150.0,...,1,0,1,None,None,9,2025-05-08T18:15:33.117272,467 days,215 days,252 days


In [38]:
HEU_TTG_TOTAL = round(df_heu_total['TTG_timedelta'].mean().total_seconds() / 86400, 2)

HEU_TTG_TOTAL

342.48